<a href="https://colab.research.google.com/github/j20nch/class2022Spring/blob/main/tensorflow_hub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### [Tensorflow Hub](https://www.tensorflow.org/hub)

models에 가면 모델들을 다 모아놨음 (text, image, sound이 3가지를 넘어가지를 않음. 그 중에서 image classification > image를 들어가서 보면 설명이 조금 추상적/단순화되어 있긴 하지만...

### image classification (e.g. [inception_v3](https://tfhub.dev/google/imagenet/inception_v3/classification/5))

아래 코드들은 대표적인 예시를 python으로 구현한 것임.

In [1]:
import tensorflow.compat.v2 as tf
import tensorflow_hub as hub

tenserflow v2를 입력(홈페이지에 주어져있던 코드)

In [2]:
model = hub.KerasLayer("https://tfhub.dev/google/imagenet/inception_v3/classification/5")

사이트 명은 어디서 가져오나 (괄호 안 내용)
아래에 inception V3: classification 눌러서 가져온 것. (Copy URL 누름)

model을 hub.KerasLayer로 해서 링크를 붙여 가져옴. 이 model 자체가 함수같이 쓰임 (pipeline과 살짝 유사)

In [3]:
from PIL import Image
import requests
url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

image를 가져온 것. PIL 형태(이미지 관련 library)의 데이터, 완전 numerical하게 쓸 수 있는 형태는 아직 아님.

In [4]:
import numpy as np
image = np.array(image)
# reshape into shape [batch_size, height, width, num_channels]
image = tf.reshape(image, [1, image.shape[0], image.shape[1], image.shape[2]])
# Use `convert_image_dtype` to convert to floats in the [0,1] range.
image = tf.image.convert_image_dtype(image, tf.float32)

image를 바로 쓸 수 있는 게 아님 > numpy로 array화를 해야 쓸 수 있음! (위 image는 (480,640,3) 형태

> 그걸 또 그대로 쓸 수 있는 게 아니라 "reshape"가 필요. 원래는 3차원의 array였는데, 4차원의 array로 바꾸어 줘야 함. (모델 자체가 변수를 4차원으로 받기 때문)
원래 그림 하나만 하려면 3차원으로 받으면 되지만, 그림을 batch로(1000개씩, 100개씩) 제공할 수 있는 것. 그래서 첫번째 항목에는 batch 크기를 쓰면 됨 (지금은 한 장이니까 1)
(1, 480, 640, 3) 형태가 나옴.

>> 맨 아래줄에 또 한 번 더 convert를 해줌. > 우리가 지금 쓰려 하는 모델 자체가 반드시 [0,1]사이로 (어두우면 0, 밝으면 1 이런식으로 되어 있을 것) 수를 요구함. 4차원의 array인데, 0에서 1사이의 숫자로 되어 있는 걸 입력으로 받아 출력을 뱉어줌.
(image[0,:,:,0]을 입력해보면 숫자들이 대부분 0에서 1 사이의 숫자로 바뀌어 있음을 확인할 수 있음.)

In [5]:
logits = model(image)
np.argmax(logits)

282

위에서 만든 모델에 image를 넣으면 output을 뱉어줌. 이 모델 안에는 행렬들이 한참 들어있다고 생각하면 되고 image는 vector라 생각하면 됨.

argmax: 몇 번째 거가 가장 높은 값인지. 결과값 282가 뭘 의미하는가는 아래 링크를 들어가보기.

> https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt

inception V3가 훈련했던 것들임. 거기 데이터들을 어떻게 만들었냐, 어떤 그림과 그 그림에 대한 label을 pair로 주면서 훈련을 시켰을 것. 그럼 여기에 사용된 label이 총 몇개가 있나? 이게 1000개가 있다는 의미. 그게 282번째 거가 확률이 제일 높더라!

> 이걸 다 카피해서 엑셀에 가서 282번째가 뭔지 확인하면 됨. (0번째부터니까 엑셀에서는 283을 찾으면 됨)